# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.
%load_ext dotenv
%dotenv 



In [ ]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [12]:
import os
from glob import glob

price_data_dir = os.getenv('PRICE_DATA')

parquet_files = glob(os.path.join(price_data_dir, "**/*.parquet/*.parquet"))

#  I had to limit number of files to 10, because it was taking too long to run.
parquet_files = parquet_files[:10]
parquet_files

['../../05_src/data/prices/CTAS/CTAS_2008.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2018.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2011.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2001.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2000.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2010.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2019.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2009.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2012.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2002.parquet/part.0.parquet']

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Write your code below.


def process_data(f_path):
    df = dd.read_parquet(f_path).set_index('ticker')

    df = df.repartition(partition_size="10MB")

    df = (df.groupby('ticker', group_keys=False).apply(
        lambda x: x.assign(Close_lag_1=x['Close'].shift(1))
    ).assign(
        Adj_Close=lambda x: x['Close']
    ).assign(
        Adj_Close_lag_1=lambda x: x['Adj_Close'].shift(1)
    ).assign(
        returns=lambda x: (x['Close'] / x['Close_lag_1']) - 1
    ).assign(
        hi_lo_range=lambda x: x['High'] - x['Low']
    ))

    return df

dd_list = [process_data(f) for f in parquet_files]

dd_f = dd.concat(dd_list)




+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [14]:
# Write your code below.


dd_f = dd_f.compute()

dd_f['rolling_avg_return'] = dd_f.groupby('ticker')['returns'].transform(lambda x: x.rolling(window=10).mean())

print(dd_f.head())  # To inspect the result

             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
CTAS   2008-01-02  33.490002  33.619999  32.259998  32.349998  25.333313   
CTAS   2008-01-03  32.610001  33.150002  32.509998  32.970001  25.818832   
CTAS   2008-01-04  32.650002  32.740002  31.610001  31.610001  24.753817   
CTAS   2008-01-07  31.620001  32.869999  31.620001  32.470001  25.427284   
CTAS   2008-01-08  32.540001  32.590000  31.100000  31.100000  24.354439   

         Volume       sector                     subsector  year  Close_lag_1  \
ticker                                                                          
CTAS    1448400  Industrials  Diversified Support Services  2008          NaN   
CTAS    1334800  Industrials  Diversified Support Services  2008    32.349998   
CTAS    1423100  Industrials  Diversified Support Services  2008    32.970001   
CTAS    1486500  Industrials  Diversified Support Services  20

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
No, it is not necessary. Dask can  handle it as well. Converting to pandas might be required if the dataset is small and fits into memory, but for larger dataset dask would be preferable.

Would it have been better to do it in Dask? Why?

Yes, dask would perform better large data sets
Pros:
Dask is able to split big data set in a smaler chanks and run it in paralel 
Dask uses lazy evaluation, which means that it builds a task graph of the operations and executes them only when needed
Keeping the computation within Dask we avoid potential issues of converting back and forth and keep workflow consistent.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.